In [1]:
import math

In [2]:
class Value:
    def __init__(self, data, _operation = "", _children = (), label = ""):
        self.data = data
        self._prev = set(_children)
        self._operation = _operation
        self.label = label
        self.grad = 0.0
        self._backward = lambda : None
        
    def __repr__(self):
        return f"Value(data = {self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        operation = " add "
        children = (self, other)
        addition = self.data + other.data
        result = Value(data = addition, _operation = operation, _children = children)

        def _backward():
            self.grad += 1.0 * result.grad
            other.grad += 1.0 * result.grad

        self._backward = _backward
            
        
        return result

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        operation = " mul "
        children = (self, other)
        addition = self.data * other.data
        result = Value(data = addition, _operation = operation, _children = children)
        
        def _backward():
            self.grad += other.data * result.grad
            other.grad += self.data * result.grad

        self._backward = _backward        
        
        return result

    def __rmul__(self, other):
        return self * other

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __neg__(self):
        return -1 * self

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "power can be integer or float"
        operation = f" pow({other})"
        children = (self, )
        power = self.data ** other
        result = Value(data = power, _children = children, _operation = operation)
        
        def _backward():
            self.grad += other * self.data ** (other - 1) * result.grad

        self._backward = _backward        
        
        return result

    def __truediv__(self, other):
        return self * (other ** -1)

    def __rtruediv__(self, other):
        return other * (self ** -1)

    def exp(self):
        operation = " exp "
        children = (self, )
        e = math.exp(self.data)
        result = Value(data = e, _operation = operation, _children = children)

        def _backward():
            self.grad += self.data * result.grad

        self._backward = _backward
        
        return result

    def tanh(self):
        x = self.data
        operation = " tanh "
        children = (self, )
        tan_h = (math.exp(2.0 * x) - 1.0) / (math.exp(2.0 * x) + 1.0)
        result = Value(data = tan_h, _operation = operation, _children = children)
        
        def _backward():
            self.grad += 1 - tan_h**2

        self._backward = _backward        
        
        return result

In [7]:
from graphviz import Digraph
def trace(root):
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges
def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    dot.node(name=uid, label="{%s | data %.4f }" % (n.label, n.data), shape="record")
    if n._operation:
      dot.node(name = uid + n._operation, label = n._operation)
      dot.edge(uid + n._operation, uid)
  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._operation)

  return dot